# Convolutional Neural Network

### Importing the libraries

In [2]:
# !pip install --upgrade tensorflow keras Pillow

In [3]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

2021-08-27 20:11:51.541012: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-27 20:11:51.541034: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8001 images belonging to 2 classes.


### Preprocessing the Test set

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [7]:
cnn = tf.keras.models.Sequential()

2021-08-27 20:11:55.062011: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-27 20:11:55.062032: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-27 20:11:55.062047: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (iurisevero-Inspiron-3583): /proc/driver/nvidia/version does not exist
2021-08-27 20:11:55.062241: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Step 1 - Convolution

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [11]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [12]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [13]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.1))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.1))

### Step 5 - Output Layer

In [14]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [15]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [17]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 50)

2021-08-27 20:12:14.080748: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
251/251 [==============================] - 32s 126ms/step - loss: 0.6830 - accuracy: 0.5458 - val_loss: 0.6453 - val_accuracy: 0.6270
Epoch 2/50
251/251 [==============================] - 33s 132ms/step - loss: 0.6261 - accuracy: 0.6518 - val_loss: 0.6042 - val_accuracy: 0.6645
Epoch 3/50
251/251 [==============================] - 33s 133ms/step - loss: 0.5852 - accuracy: 0.6908 - val_loss: 0.5597 - val_accuracy: 0.7240
Epoch 4/50
251/251 [==============================] - 33s 132ms/step - loss: 0.5575 - accuracy: 0.7158 - val_loss: 0.5636 - val_accuracy: 0.7265
Epoch 5/50
251/251 [==============================] - 33s 133ms/step - loss: 0.5355 - accuracy: 0.7303 - val_loss: 0.4936 - val_accuracy: 0.7590
Epoch 6/50
251/251 [==============================] - 34s 134ms/step - loss: 0.5134 - accuracy: 0.7463 - val_loss: 0.4679 - val_accuracy: 0.7855
Epoch 7/50
251/251 [==============================] - 33s 133ms/step - loss: 0.4865 - accuracy: 0.7695 - val_loss: 0.4971 - val_ac

## Part 4 - Making a single prediction

In [18]:
import numpy as np
from keras.preprocessing import image
prediction = []
for i in range(1,3):
    test_image = image.load_img(f'dataset/single_prediction/cat_or_dog_{i}.jpg', target_size = (64, 64))
    print(f'dataset/single_prediction/cat_or_dog_{i}.jpg')
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    print(training_set.class_indices)
    print(result, result[0][0])
    if result[0][0] == 1:
      prediction.append('dog')
    else:
      prediction.append('cat')

dataset/single_prediction/cat_or_dog_1.jpg
{'cats': 0, 'dogs': 1}
[[1.]] 1.0
dataset/single_prediction/cat_or_dog_2.jpg
{'cats': 0, 'dogs': 1}
[[7.948744e-10]] 7.948744e-10


In [ ]:
print(prediction)

In [ ]:
test_image = image.load_img(f'dataset/training_set/cats/cat.1005.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(training_set.class_indices)
print(result, result[0][0])

## Fim